# 네이버 매장 -> 매장에 있는 tag 

문제
 
1. 추천순으로 긁어올지, 최신순으로 긁어올지

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
from tqdm import tqdm
from bs4 import BeautifulSoup

import datetime
import time
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import requests
from openpyxl import Workbook
from bs4 import BeautifulSoup
import pandas as pd

불러오기 할 때 조건으로 리뷰 n개 이상만 가져오게 수정 필요

In [2]:
# 데이터 불러오기
original_res_df = pd.read_excel('combined_file.xlsx')
original_df = original_res_df.copy()
original_df = original_df.loc[original_df['검색여부'] == '가능'].reset_index(drop=True)
original_df.drop('Unnamed: 0', axis=1, inplace=True)


# 컬럼 순서 바꾸기 -> 보기 좋으라고 바꿈
original_df['검색어'] = original_df['상세주소'] + " " + original_df['사업장명']
column_order = ['검색어', '업태구분명', '사업장명', '사업장명_원본', '시도', '지역구', '상세주소', '소재지전체주소', '도로명전체주소', '좌표정보(x)', '좌표정보(y)', '검색여부', '영업상태명']
original_df = original_df[column_order]
original_df.tail()

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
8291,"도곡로 401, 간코",일식,간코,간코 롯데백화점강남점,서울특별시,강남구,"도곡로 401,",서울특별시 강남구 대치동 937 롯데백화점,"서울특별시 강남구 도곡로 401, 롯데백화점 지하1층 (대치동)",204669.543367,443873.621189,가능,영업/정상
8292,"압구정로 108, 오토김밥",분식,오토김밥,오토김밥 신사점,서울특별시,강남구,"압구정로 108,",서울특별시 강남구 신사동 528-3 덕산빌딩,"서울특별시 강남구 압구정로 108, 덕산빌딩 지상1층 10호 (신사동)",201749.456414,446742.828124,가능,영업/정상
8293,"압구정로42길 36, 마들마들",한식,마들마들,마들마들,서울특별시,강남구,"압구정로42길 36,",서울특별시 강남구 신사동 635-14 금광아크존1,"서울특별시 강남구 압구정로42길 36, 지상1층 101호 (신사동, 금광아크존1)",202990.870783,447146.555568,가능,영업/정상
8294,"언주로151길 21, 세바",경양식,세바,세바(Sevva),서울특별시,강남구,"언주로151길 21,",서울특별시 강남구 신사동 629-1,"서울특별시 강남구 언주로151길 21, 지상1층 (신사동)",202731.421127,446609.460134,가능,영업/정상
8295,"논현로161길 37, 구구구",경양식,구구구,구구구 (GOOGOOGOO),서울특별시,강남구,"논현로161길 37,",서울특별시 강남구 신사동 552-5,"서울특별시 강남구 논현로161길 37, 동쪽 지상1층 (신사동)",202153.410454,446748.562578,가능,영업/정상


In [3]:
# original_df.to_excel('./data/res_original_df.xlsx')

### tag 데이터 크롤링

In [4]:
# start_num = 0 # 할 차례
# num = 100
# end_num = start_num + num
# res_df = original_df[start_num:end_num]
# res_df = res_df.reset_index(drop=True)
# res_df.head(3)

In [5]:
# # # 해민
# # import chromedriver_autoinstaller
# # chromedriver_autoinstaller.install()
# # driver = webdriver.Chrome()

# # 지선
# driver = webdriver.Chrome(ChromeDriverManager().install())

# # 빈 리스트 생성
# restaurant_name_list = []
# category_name_list = []
# price_list = []

# for i in tqdm(range(0, len(res_df)), desc='진행 중', position=0, leave=True):
#     name = res_df['검색어'][i]
#     driver.get('https://map.naver.com/p/search/{}'.format(name))
#     time.sleep(3)  

#     try :
#         if driver.find_elements(By.ID,'entryIframe') :
#             entryIframe = driver.find_element(By.ID,'entryIframe')
#             driver.switch_to.frame(entryIframe)
#     except :
#         pass 

#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')

#     try : 
#         # '리뷰' 탭의 href 속성 가져오기
#         review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
#         review_url = 'https://pcmap.place.naver.com'+review_tab_href

#         driver.get(review_url)
#         time.sleep(2.2)  

#         for _ in range(4):
#         # 스크롤을 위한 JavaScript 코드 실행
#             driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(2.5) 

#         # HTML 추출
#         html = driver.page_source
#         soup = BeautifulSoup(html, 'html.parser')
#         items = soup.select('ul.uNsI9 li.nbD78, ul.uNsI9 li.nbD78.Xcpsu')

#         # 여러 식당 정보를 반복하며 리스트에 추가
#         for item in items:
#             restaurant_name_list.append(name)
#             category_name_list.append(item.text.split('"')[1])
#             price_list.append(int(''.join(filter(str.isdigit, item.text))))

#         data_list = {
#             '매장명': restaurant_name_list,
#             'tag': category_name_list,
#             '인원': price_list
#         }
#     except :
#         data_list = {
#             '매장명': restaurant_name_list,
#             'tag': '리뷰 10개 미만',
#             '인원': ''
#         }
#         continue

# result_df = pd.DataFrame(data_list)
# result_df # 100개당 22분 
# result_df.to_excel(f'./data/restaurant_tag_data/restaurant_tag_df_{start_num}_{end_num}.xlsx', index=False)

### 리뷰 데이터 크롤링

In [6]:
start_num = 0 # 할 차례
num = 5
end_num = start_num + num
res_df = original_df[start_num:end_num]
new_res_df_test = res_df.reset_index(drop=True)
new_res_df_test

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
0,"강남대로114길 18, 정식당",한식,정식당,정식당,서울특별시,강남구,"강남대로114길 18,",서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",202203.971845,444853.761458,가능,영업/정상
1,"강남대로 378, 채선당 샤브보트",한식,채선당 샤브보트,채선당 샤브보트 강남역점,서울특별시,강남구,"강남대로 378,",서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",202462.794327,443827.961945,가능,영업/정상
2,"압구정로 343, 수아당",한식,수아당,수아당,서울특별시,강남구,"압구정로 343,",서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",203470.848439,447369.579852,가능,영업/정상
3,"역삼로 204, 알찬한끼",분식,알찬한끼,알찬한끼,서울특별시,강남구,"역삼로 204,",서울특별시 강남구 역삼동 769-1 삼환 아르누보,"서울특별시 강남구 역삼로 204, 삼환 아르누보 지하1층 B09,B10호 중 107...",203453.074369,443736.685792,가능,영업/정상
4,"삼성로96길 11, 세계관",한식,세계관,세계관,서울특별시,강남구,"삼성로96길 11,",서울특별시 강남구 삼성동 155-11,"서울특별시 강남구 삼성로96길 11, 지상2층 (삼성동)",204952.006872,445195.026116,가능,영업/정상


# 음식점 리뷰 크롤링

In [14]:
# 빈 리스트 생성
user_id_list = []
content_list = []
date_list = []  
revisit_list = []
tag_category_name_list = []
restaurant_name_list = []
review_category_name_list = []

driver = webdriver.Chrome(ChromeDriverManager().install())

scroll_num = 5

for i in range(0,len(new_res_df_test)):
  
    name = new_res_df_test['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

    time.sleep(3)  
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
    except :
        pass 

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try : 
        # '리뷰' 탭의 href 속성 가져오기
        review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
        review_url = 'https://pcmap.place.naver.com'+review_tab_href

        driver.get(review_url)
        time.sleep(2.2)  

    except :
        print('끝')

    # 현재 페이지 URL 가져오기
    current_url = driver.current_url

    try : 
        # 리뷰 url로 이동 
        modified_url = current_url.replace('/home', '/review/visitor')
        driver.get(modified_url)
    except :
        continue

    # 리뷰 스크래핑 시작
    try:
        # 스크롤
        for _ in range(scroll_num):
            current_scroll_position = driver.execute_script('return window.scrollY;') # 초기 스크롤 위치 

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 
            
            # 'TeItc' 클래스를 가진 요소가 클릭 가능할 때까지 기다리고, 보이도록 스크롤
            button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'TeItc')))
            driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

            # 더보기 클릭
            button = driver.find_element_by_class_name('fvwqf')
            button.click()

            # 'fvwqf' 클래스를 가진 요소가 나타날 때까지 스크롤
            driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

            # ------------ 리뷰, 태그 '내용 더보기' 버튼 클릭 ----------------------
            review_buttons = driver.find_elements_by_class_name('xHaT3')  
            for button1 in review_buttons:
                try:
                    button1.click()
                except :
                    pass

            tag_buttons = driver.find_elements_by_css_selector('.gyAGI a.P1zUJ.ZGKcF')  
            for button2 in tag_buttons:
                try:
                    button2.click()
                except :
                    pass
            # ----------------------------------------------
            time.sleep(3)

            # ---------------스크롤위치가 변하지 않으면 리뷰를 다 크롤링한 것이니까 탈출 -------------------------
            # 현재 스크롤 
            new_scroll_position = driver.execute_script('return window.scrollY;')

            # 현재 위치와 이전 위치가 같으면 반복문 탈출
            if new_scroll_position == current_scroll_position:
                break

            # 이전 스크롤 위치 업데이트
            prev_scroll_position = current_scroll_position
            # -------------------------------------------------------------------------------------------------
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
            reviews = soup.select('li.owAeM') 

        # 리뷰 크롤링 시작
        for r in reviews:
            user_id = r.select_one('div.qgLL3 span.P9EZi')
            review_count = r.select_one('div.Gt2_9 span.RNn6x:nth-of-type(1)')
            photo_count = r.select_one('div.Gt2_9 span.RNn6x:nth-of-type(2)')
            
            content = r.select_one('div.vg7Fp span.zPfVt')
            
            try:
                tag = r.select_one('div.ERkm0 span.sIv5s')
            except:
                tag = ""
            
            date = r.select_one('div.jxc2b div.D40bm span.CKUdu time')
            revisit = r.select_one('div.jxc2b div.D40bm span.CKUdu:nth-of-type(2)')
            
            # 없는 경우 
            user_id_text = user_id.text if user_id else ''
            review_count_text = review_count.text if review_count else ''
            photo_count_text = photo_count.text if photo_count else ''
            content_text = content.text if content else ''
            tag_text = tag.text if tag else ''
            date_text = date.text if date else ''
            revisit_text = revisit.text.replace('번째 방문', '') if revisit else ''
            
            # 리스트에 합치기
            restaurant_name_list.append(name)
            user_id_list.append(user_id_text)
            review_category_name_list.append(tag_text)
            date_list.append(date_text)
            revisit_list.append(revisit_text)
            content_list.append(content_text)

    except Exception as e:
        print(e)


# 데이터프레임 얹히기
review_data_list = {
    '검색어': restaurant_name_list,
    'user_id': user_id_list,
    'review': content_list,
    'date': date_list,
    'revisit': revisit_list,
    'tag': review_category_name_list
}

# 리뷰 데이터 프레임 완성
review_result_df = pd.DataFrame(review_data_list)

review_result_df

Message: no such element: Unable to locate element: {"method":"css selector","selector":".fvwqf"}
  (Session info: chrome=122.0.6261.112)



,검색어,user_id,review,date,revisit,tag
0,"강남대로114길 18, 정식당",hsy****,친구랑 어디갈까 고민하다 오게됬어요~ 근데 사진엔 없지만 불고기 진짜 미쳤구요.. ...,2.29.목,1,음식이 맛있어요
1,"강남대로114길 18, 정식당",arias7932,신논현 찐 맛집!!~오픈한지 얼마 안되었다기에 여자3명이 방문~요즘 물가가 많이 올...,1.29.월,1,음식이 맛있어요
2,"강남대로114길 18, 정식당",퍼플리보,지인 추천으로 가게된 정식당 메뉴 보고 꽃게탕과 불고기 껍데기 시켰는데 진짜…하나하...,2.29.목,1,음식이 맛있어요
3,"강남대로114길 18, 정식당",푸리타,친구소개로 2차로 들렸는데\n메뉴가 다 맛있어서\n다이어트 실패했어요 😱\n돼지껍데...,1.31.수,1,음식이 맛있어요
4,"강남대로114길 18, 정식당",행복한오늘14,회사근처여서 우연히 들렸는데 찐맛집을 발견했네요~ 고급스런 한정식집에서 파는것 같은...,1.27.토,1,친절해요
...,...,...,...,...,...,...
136,"삼성로96길 11, 세계관",심심해6739,정말 맛있었어요,23.12.30.토,1,음식이 맛있어요
137,"삼성로96길 11, 세계관",바사님,,23.12.18.월,1,음식이 맛있어요
138,"삼성로96길 11, 세계관",Bettyull,맛있어요,23.12.7.목,1,음식이 맛있어요
139,"삼성로96길 11, 세계관",난이3489,예전보다 양이 적어진것 같아요,2.5.월,1,기본 안주가 좋아요


In [15]:
review_result_df.to_excel(f'./data/restaurant_review_data/restaurant_review_df_{start_num}_{end_num}.xlsx', index=False)

# test : 한 번에 tag와 review data를 가져오고 싶은데 마음대로 안됨...